In [1]:
import io
import os
import cv2
!pip install azure-cognitiveservices-speech azure-ai-translation-document azure-core
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.environ["AZURE_SPEECH_KEY"] = "2ff3a4a0470149f8b1898c0036eea110"
os.environ["AZURE_SPEECH_REGION"] = "eastasia"

os.environ["AZURE_TRANSLATOR_KEY"] = "1b2924ec518c471e98dfe473aaba0adb"
os.environ["AZURE_TRANSLATOR_ENDPOINT"] = "https://api.cognitive.microsofttranslator.com/"



In [4]:
!pip install sounddevice scipy
!apt-get install libportaudio2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 0s (174 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/l

In [5]:
from IPython.display import Javascript, display, Audio as IPythonAudio
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
!pip install -q pyspellchecker
from pydub import AudioSegment



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.0 MB/s eta 0:00:00


In [6]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

In [7]:
'''audio = record(5)
audio.export("/content/drive/MyDrive/Audio_Input/Audio_Sample.wav", format="wav", parameters=["-b:a", "16"])

IPythonAudio("/content/drive/MyDrive/Audio_Input/Audio_Sample.wav", autoplay=True)'''


'audio = record(5)\naudio.export("/content/drive/MyDrive/Audio_Input/Audio_Sample.wav", format="wav", parameters=["-b:a", "16"])\n\nIPythonAudio("/content/drive/MyDrive/Audio_Input/Audio_Sample.wav", autoplay=True)'

In [8]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_audio_to_text(audio_path):
    speech_key = os.environ.get("AZURE_SPEECH_KEY")
    speech_region = os.environ.get("AZURE_SPEECH_REGION")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = "gu-IN"

    audio_config = speechsdk.audio.AudioConfig(filename=audio_path)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    result = recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    else:
        return None

audio_path ='/content/drive/MyDrive/Audio_Input/Audio_Sample.wav'
gujarati_text= transcribe_audio_to_text(audio_path)
print("Gujarati Text: ", gujarati_text)


Gujarati Text:  અનિનીમાં ઉપરી દિબાંગ ઘાટી જિલ્લાનું મુખ્યાલય છે.


In [9]:
import requests

def translate_text(text, to_lang="en"):
    subscription_key = os.environ.get("AZURE_TRANSLATOR_KEY")
    endpoint = os.environ.get("AZURE_TRANSLATOR_ENDPOINT") + "/translate"

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': os.environ.get("AZURE_SPEECH_REGION"),
        'Content-type': 'application/json'
    }

    params = {
        'api-version': '3.0',
        'from': 'gu',
        'to': [to_lang]
    }

    body = [{
        'text': text
    }]

    request = requests.post(endpoint, headers=headers, params=params, json=body)
    response = request.json()

    return response[0]['translations'][0]['text']

english_text = translate_text(gujarati_text)
print("Translated English Text: ", english_text)


Translated English Text:  Anini is the headquarters of upper Dibang Valley district.


In [10]:
!pip install stanza
!pip install -q moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 30.3 MB/s eta 0:00:00


In [11]:
import os
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytor

In [12]:
import stanza

def parse(text):
    doc = nlp(text)
    result = []
    for sentence in doc.sentences:
        for word in sentence.words:
            result.append((word.text.lower(), word.lemma.lower()))
    return result
result = parse(english_text)
print(result)

resultant = []
for word in result:
  resultant.append(word[1])
import os
folder = os.getcwd()
filePrefix = folder + "/drive/MyDrive/Sign_language_dataset"


print(filePrefix)

files = [filePrefix +"/"+ word.lower() + ".mp4" for word in resultant]
print(files)
existing_files = [file for file in files if os.path.exists(file)]
print(existing_files)

[('anini', 'anini'), ('is', 'be'), ('the', 'the'), ('headquarters', 'headquarters'), ('of', 'of'), ('upper', 'upper'), ('dibang', 'dibang'), ('valley', 'valley'), ('district', 'district'), ('.', '.')]
/content/drive/MyDrive/Sign_language_dataset
['/content/drive/MyDrive/Sign_language_dataset/anini.mp4', '/content/drive/MyDrive/Sign_language_dataset/be.mp4', '/content/drive/MyDrive/Sign_language_dataset/the.mp4', '/content/drive/MyDrive/Sign_language_dataset/headquarters.mp4', '/content/drive/MyDrive/Sign_language_dataset/of.mp4', '/content/drive/MyDrive/Sign_language_dataset/upper.mp4', '/content/drive/MyDrive/Sign_language_dataset/dibang.mp4', '/content/drive/MyDrive/Sign_language_dataset/valley.mp4', '/content/drive/MyDrive/Sign_language_dataset/district.mp4', '/content/drive/MyDrive/Sign_language_dataset/..mp4']
['/content/drive/MyDrive/Sign_language_dataset/be.mp4', '/content/drive/MyDrive/Sign_language_dataset/the.mp4', '/content/drive/MyDrive/Sign_language_dataset/of.mp4', '/cont

In [13]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
from IPython.display import HTML
from base64 import b64encode


def trim_video(clip, trim_duration=0.0001):
    new_duration = clip.duration - trim_duration
    if new_duration > 0:
        return clip.subclip(0, new_duration)
    else:
        return clip

clips = [trim_video(VideoFileClip(file)) for file in existing_files]


final_clip = concatenate_videoclips(clips)


output_path = '/content/drive/MyDrive/output_video4.mp4'
final_clip.write_videofile(output_path)


def display_video(path, autoplay=False):
    video_file = open(path, "rb").read()
    video_url = "data:video/mp4;base64," + b64encode(video_file).decode()
    autoplay_attr = "autoplay" if autoplay else ""
    return HTML(f"""
    <video width="640" height="360" {autoplay_attr} controls>
        <source src="{video_url}" type="video/mp4">
    </video>
    """)
display_video(output_path,autoplay="True")


Moviepy - Building video /content/drive/MyDrive/output_video4.mp4.
Moviepy - Writing video /content/drive/MyDrive/output_video4.mp4



t:  25%|██▍       | 46/185 [00:00<00:01, 83.39it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/Sign_language_dataset/be.mp4, 691200 bytes wanted but 0 bytes read,at frame 46/48, at time 1.53/1.57 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  48%|████▊     | 89/185 [00:02<00:02, 32.04it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/Sign_language_dataset/the.mp4, 691200 bytes wanted but 0 bytes read,at frame 44/45, at time 1.47/1.50 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  99%|█████████▉| 183/185 [00:05<00:00, 42.34it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-pa

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/output_video4.mp4


In [14]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [ ]:
import gradio as gr
import os
import azure.cognitiveservices.speech as speechsdk
from moviepy.editor import VideoFileClip, concatenate_videoclips
from spellchecker import SpellChecker
import stanza
import requests

# Initialize spell checker and stanza
spell = SpellChecker()
stanza.download('en')
nlp = stanza.Pipeline('en')

# Function to transcribe Gujarati audio to text using Azure Cognitive Services
def transcribe_audio_to_text(audio_path):
    speech_key = os.getenv("AZURE_SPEECH_KEY")
    speech_region = os.getenv("AZURE_SPEECH_REGION")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = "gu-IN"

    audio_config = speechsdk.audio.AudioConfig(filename=audio_path)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    result = recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    else:
        return None

# Function to translate text using Azure Translator
def translate_text(text, to_lang="en"):
    subscription_key = os.getenv("AZURE_TRANSLATOR_KEY")
    endpoint = os.getenv("AZURE_TRANSLATOR_ENDPOINT") + "/translate"

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': os.getenv("AZURE_SPEECH_REGION"),
        'Content-type': 'application/json'
    }

    params = {
        'api-version': '3.0',
        'from': 'gu',
        'to': [to_lang]
    }

    body = [{'text': text}]

    response = requests.post(endpoint, headers=headers, params=params, json=body)
    response = response.json()

    return response[0]['translations'][0]['text']

# Function to parse text using Stanza
def parse(text):
    doc = nlp(text)
    result = [(word.text.lower(), word.lemma.lower()) for sentence in doc.sentences for word in sentence.words]
    return result

# Function to generate video from parsed words
def generate_video(word_list):
    folder = os.getcwd()
    filePrefix = folder + "/drive/MyDrive/Sign_language_dataset"
    files = [filePrefix + "/" + word + ".mp4" for word in word_list]
    existing_files = [file for file in files if os.path.exists(file)]

    if not existing_files:
        raise FileNotFoundError("No video clips found for the given words.")

    try:
        clips = [VideoFileClip(file) for file in existing_files]
        final_clip = concatenate_videoclips(clips)
        output_path = '/content/drive/MyDrive/g.mp4'
        final_clip.write_videofile(output_path)
    except Exception as e:
        return f"Error during video concatenation: {e}"

    return output_path

# Function to process the audio file and generate a sign language video
def process_audio(audio_file):
    try:
        gujarati_text = transcribe_audio_to_text(audio_file)
        if not gujarati_text:
            return None, "Error: Could not transcribe audio."

        english_text = translate_text(gujarati_text)
        if not english_text:
            return None, "Error: Could not translate text."

        parsed = parse(english_text)
        word_list = [word[1] for word in parsed]
        video_path = generate_video(word_list)

        if not video_path:
            return None, "Error: No matching video clips found."

        return video_path, ""  # Return video path on success with an empty error message
    except Exception as e:
        return None, f"Error processing audio file: {e}"  # Return error message

# Gradio Interface Setup
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Video(label="Generated Sign Language Video"), gr.Textbox(label="Error Message")],
    title="Gujarati Audio to Sign Language",
    description="Upload a Gujarati audio file to receive a corresponding sign language video."
)

# Launch the interface with sharing and debugging enabled
iface.launch(share=True, debug=True)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
  checkpoint = torch.load(filename, lambda storage, loc: storage)

INFO:stanza:Loading: mwt
  checkpoint = torch.load(filename, lambda storage, loc: storage)

INFO:stanza:Loading: pos
  checkpoint = torch.load(filename, lambda storage, loc: storage)

  data = torch.load(self.filename, lambda storage, loc: storage)

  state = torch.load(filename, lambda stor

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7e50a421fc98256b0d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
